In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd/content/drive/MyDrive/Classroom/KY4/DAP/API_DB2_IBM

/content/drive/MyDrive/Classroom/KY4/DAP/API_DB2_IBM


In [3]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://hebpm48hhaj-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [4]:
!pip install ibm_db

In [5]:
import ibm_db
import os
import pandas as pd

# Set your IBM Db2 credentials in the environment variables or manually assign them in the code below.
dsn_hostname = os.getenv('DB_HOST', '__change this___') # change this
dsn_uid = os.getenv('DB_UID', '__change this___')  # change this
dsn_pwd = os.getenv('DB_PWD', '__change this___')  # change this
dsn_port = os.getenv('DB_PORT', '__change this___')  # change this
dsn_database = "bludb"  # change if necessary
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_protocol = "TCPIP"
dsn_security = "SSL"


# Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd, dsn_security)

# Create database connection
try:
    conn = ibm_db.connect(dsn, "", "")
    print("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

    # Query to get the first 5 rows from the table
    command = "SELECT * FROM STUDENTS FETCH FIRST 1 ROWS ONLY"   # Câu lệnh SQL command = "SELECT * FROM cb106 FETCH FIRST 1 ROWS ONLY" đơn giản là truy vấn để lấy một dòng dữ liệu đầu tiên từ bảng cb106 trong cơ sở dữ liệu.
                                                              # Đây là một cú pháp phổ biến trong SQL để giới hạn số lượng hàng trả về.
    # SELECT column_name FROM cb106 FETCH FIRST 1 ROWS ONLY;  # Lấy cột column_name hàng 1

    stmt = ibm_db.exec_immediate(conn, command)

    # Fetch the result set
    result = ibm_db.fetch_both(stmt)
    rows = []

    # Loop through the result and fetch the rows
    while result:
        rows.append(result)
        result = ibm_db.fetch_both(stmt)

    # Convert result to a pandas DataFrame for better readability
    df = pd.DataFrame(rows)
    print(df)

except Exception as e:
    print("Error connecting to the database: ", str(e))
finally:
    # Close the connection
    if conn:
        ibm_db.close(conn)


Connected to database:  bludb as user:  ptg18620 on host:  55fbc997-9266-4331-afd3-888b05e734c0.bs2io90l08kqb1od8lcg.databases.appdomain.cloud
  TEN_SINH_VIEN             0   NGAY_SINH           1  MA_SV      2  \
0  Nguyễn Văn A  Nguyễn Văn A  1999-01-15  1999-01-15  SV001  SV001   

          CHUYEN_NGANH                    3  
0  Công nghệ thông tin  Công nghệ thông tin  


In [9]:
from flask import Flask, render_template, request, redirect, url_for
import ibm_db

app = Flask(__name__)

# Chuỗi kết nối đến DB2
conn_str = (
    "DATABASE=bludb;"
    "HOSTNAME=__change this___;"
    "PORT=__change this___;"
    "PROTOCOL=TCPIP;"
    "UID=__change this___;"
    "PWD=__change this___;"
    "SECURITY=SSL;"
)

# Cố gắng kết nối đến DB2 và xử lý lỗi nếu có
try:
    conn = ibm_db.connect(conn_str, "", "")
    print("Kết nối thành công đến DB2!")
except Exception as e:
    print(f"Không thể kết nối đến DB2: {str(e)}")
    sys.exit(1)  # Thoát chương trình nếu không kết nối được

# Trang chính hiển thị danh sách sinh viên
@app.route('/')
def home():
    sql = "SELECT * FROM Students"
    stmt = ibm_db.exec_immediate(conn, sql)
    students = []
    row = ibm_db.fetch_assoc(stmt)
    while row:
        students.append(row)
        row = ibm_db.fetch_assoc(stmt)
    return render_template('index.html', students=students)

# Hàm thêm sinh viên vào DB2
@app.route('/add', methods=['POST'])
def add_student():
    name = request.form['name']
    dob = request.form['dob']
    student_id = request.form['student_id']
    major = request.form['major']

    # SQL Insert
    sql = "INSERT INTO Students (Ten_Sinh_Vien, Ngay_Sinh, Ma_SV, Chuyen_Nganh) VALUES (?, ?, ?, ?)"
    stmt = ibm_db.prepare(conn, sql)
    ibm_db.bind_param(stmt, 1, name)
    ibm_db.bind_param(stmt, 2, dob)
    ibm_db.bind_param(stmt, 3, student_id)
    ibm_db.bind_param(stmt, 4, major)
    ibm_db.execute(stmt)

    return redirect(url_for('home'))

# Hàm xóa sinh viên khỏi DB2
@app.route('/delete', methods=['POST'])
def delete_student():
    student_id = request.form['student_id']

    # SQL Delete
    sql = "DELETE FROM Students WHERE Ma_SV = ?"
    stmt = ibm_db.prepare(conn, sql)
    ibm_db.bind_param(stmt, 1, student_id)
    ibm_db.execute(stmt)

    return redirect(url_for('home'))

if __name__ == '__main__':
    app.run(port=5000, debug=True)


Kết nối thành công đến DB2!
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
